<h1>Install Dependcies</h1>

In [1]:
!pip install torch
!pip install numpy
!pip install gymnasium
!pip install stable_baselines3[extra]


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached stable_baselines3-2.1.0-py3-none-any.whl (178 kB)



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<h1>Imports</h1>

In [2]:
import gymnasium as gym
from gym import Env
from gym.spaces import Discrete,MultiDiscrete, Box
import numpy as np

ModuleNotFoundError: No module named 'gym'

<h1>Create Game Classes</h1>

In [1]:
# %load classes.py
import functools
import random
class Domino():
    def __init__(self, s1:int,s2:int):
        self.sides = (s1,s2)
        self.isDouble = (s1 == s2)
    def evalute_side(self, side: int):
        if(self.sides[0] == side): return 0
        elif (self.sides[1] == side): return 1
    def calc_points(self):
        return self.sides[0] + self.sides[1]
    def __str__(self):
        return str(self.sides)


class BoneYard():
    def __init__(self):
        self.dominos = []
        self.build()
        self.shuffle()
    def build(self):
        for i in range(0,13):
            for j in range(i,13):
                self.dominos.append(Domino(i,j))
    def shuffle(self):
        random.shuffle(self.dominos)
    def draw(self):
        if len(self.dominos) == 0: return False
        return self.dominos.pop()


class Train():
    startingSide = 12
    def __init__(self, id):
        self.openSides = [Train.startingSide]
        self.trainUp = False
        self.id = id
    def add(self,placement,domino):
        placeIndex = self.openSides.index(placement)
        trainSide = self.openSides[placeIndex]
        if(trainSide in domino.sides):
            if(domino.isDouble):
                self.openSides.pop(placeIndex)
                self.openSides.append(domino.sides[0])
                self.openSides.append(domino.sides[1])
            else:
                self.openSides.pop(placeIndex)
                self.openSides.append(domino.sides[1-domino.evalute_side(trainSide)])
        else:
            return False
    def __str__(self):
        return f"id: {self.id} trainUp?:{self.trainUp} openSides: {self.openSides}"
        

class Player():
    handSize = 12
    nextID = 0 
    def __init__(self,boneYard:BoneYard):
        self.id = Player.nextID
        Player.nextID +=1
        self.hand = []
        for _ in range(Player.handSize): self.hand.append(boneYard.draw()) 
    def highestDouble(self):
        highest = -1
        for domino in self.hand:
            if(domino.isDouble and domino.sides[0] > highest): highest = domino.sides[0]
        return highest
    def intializeTrain(self):
        if not hasattr(self,"train"):
            self.train = Train(self.id)
        return self.train
    def getDominoFromSides(self,s1:int,s2:int):
        for domino in self.hand:
            if (domino.sides == (s1,s2) or domino.sides == (s2,s1)):
                return domino
    def play(self, domino:Domino, placement:int, train:Train|None=None,firstDouble:bool=False):
        selfTrain = False
        played = False
        if train is None: 
            train = self.train
            selfTrain = True
        if firstDouble: 
            self.hand.remove(domino)
            played = True
        elif train.add(placement, domino) != False:
            self.hand.remove(domino)
            if selfTrain: self.train.trainUp = False
            played = True
        if len(self.hand) <= 0: return None
        else: return played
        

    def pointsInHand(self):
        return functools.reduce(lambda acc, domino: acc + domino.calc_points(), self.hand, 0)
    def pickup(self,boneYard:BoneYard):
        domino = boneYard.draw()
        if domino is not False:
            self.hand.append(domino)
        return domino
    def __str__(self):
        return f"id:{self.id} train:{self.train.id}"
    



class Game():
    def __init__(self,numPlayers:int):
        self.boneyard = BoneYard()
        self.players = []
        self.trains = []
        self.done = False
        self.numPlayers = numPlayers
        self.unsastifiedDouble = None
        if(numPlayers<= 4): Player.handSize = 15
        elif(numPlayers<=6): Player.handSize = 12
        elif(numPlayers<=8): Player.handSize = 10
        Player.nextID = 0
        for _ in range(numPlayers): 
            self.players.append(Player(self.boneyard))
        doubles = [player.highestDouble() for player in self.players]
        highestDouble = max(doubles)
        firstPlayer = doubles.index(highestDouble)
        firstDomino = self.players[firstPlayer].getDominoFromSides(highestDouble,highestDouble)
        for player in self.players: 
            player.intializeTrain()
            self.trains.append(player.train)
        if (len(self.players)<8): self.mexican = Train(8)
        self.centerDouble = max(doubles)
        Train.startingSide = self.centerDouble
        self.players[firstPlayer].play(firstDomino,0,firstDouble=True) #removing first double
        self.startingPlayer = self.nextPlayer(firstPlayer)
        self.prevPlayer = None
    def getTrain(self,id:int):
        if id == 8: return self.mexican
        else:
            for t in self.trains:
                if t.id==id:
                    return t
            else:
                return None
    def getPlayer(self,id:int):
        if id > 7: return None
        else: 
            for p in self.players:
                if p.id==id:
                    return p
            else:
                return None
    def nextPlayer(self, currPlayer:int):
        next = currPlayer+1 
        if (next>=self.numPlayers): next = 0 # looping around if its not an actual player
        return next

class BoardState():
    def __init__(self, trains:list[Train],centerDouble:int, mexican:Train|None = None,unsastifiedDouble=None):
        self.mexican = mexican
        self.trains = trains
        self.unsastifiedDouble = unsastifiedDouble
    #train up returns only sides that are on trains with thier trains up
    #maybe this signature should be changed to just take a list of trains? and let caller deal with filtering?
    def getPlacements(self, trainUp: bool=False,include:list[Train]=[], exclude:list[Train]=[]):
        trains = [*self.trains,self.mexican]
        placements = []
        if trainUp:
            for train in trains:
                if (train in include or train.trainUp) and train not in exclude:
                    for side in train.openSides: 
                        placements.append((train.id,side))
        else:
            for train in trains: 
                if train not in exclude:
                    for side in train.openSides: placements.append((train.id,side))
        return placements
    def getTrain(self, id):
        for train in self.trains:
            if train.id == id: return train
    def availablePlays(self, player:Player,placements:list|None=None):
        plays = []
        places = []
        if self.unsastifiedDouble is not None:
            places = [self.unsastifiedDouble]
        elif placements is not None:
            places = placements
        elif player.train.trainUp:
            places = [(player.id, side) for side in player.train.openSides]
        else: places = self.getPlacements(trainUp=True, include=[player.train])
        for placement in places:
            for domino in player.hand:
                eval = domino.evalute_side(placement[1])
                if( eval is not None): plays.append((domino.sides, placement))
        return plays
    def isValidPlay(self, player:Player, action:list[list]):
        valid = False
        plays = self.availablePlays(player)
        for play in plays:
            tuplist = [tuple(list) for list in action]
            if tuple(tuplist) == play:
                valid = True
                print(f"valid play: {play}")
        return valid
    @staticmethod
    def fromGame(game:Game):
        return BoardState(game.trains,game.centerDouble, game.mexican,game.unsastifiedDouble)


<h2>Test Game Objects</h2>

<h3>Testing intialization</h3>

In [31]:
game = Game(6)
bs = BoardState.fromGame(game)
print([str(player) for player in game.players])
print([str(domino) for domino in game.players[0].hand])
print(Train.startingSide)
plays = bs.availablePlays(game.players[0])
print(plays)


['id:0 train:0', 'id:1 train:1', 'id:2 train:2', 'id:3 train:3', 'id:4 train:4', 'id:5 train:5']
['(1, 3)', '(3, 10)', '(2, 5)', '(0, 1)', '(0, 6)', '(7, 10)', '(4, 4)', '(9, 11)', '(4, 11)', '(8, 8)', '(10, 12)', '(6, 8)']
11
[((9, 11), (0, 11)), ((4, 11), (0, 11))]


<h3>Testing random play</h3>

In [43]:
plays = bs.availablePlays(game.players[0])
if len(plays)> 0:
    play = plays[random.randint(0, len(plays) - 1)] # play = (dominoSideTuple,placementTuple)
    player = game.getPlayer(0)
    domino = player.getDominoFromSides(*play[0])
    train = bs.getTrain(play[1][0]) # play[1] is placement, = (train.id, side)
    print(train)
    player.play(domino,play[1][1],train)
    print(train)
else:
    print("no possible plays")

id: 0 trainUp?:False openSides: [12]
id: 0 trainUp?:False openSides: [4]


<h1>Start Making Env</h1>

In [4]:
# %load DominoEnv.py
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Dict, Discrete , MultiDiscrete, Box, Sequence, utils
import numpy as np
class DominoTrainEnv(Env):
    def __init__(self,numPlayers:int):
        # Actions we can take, 13,13 for possible domino sides, [9,13] for possible domino placements
        #action space needed to be changed as it would cause an unknown issue with 
        #self.action_space = MultiDiscrete(np.array([[13, 13], [9, 13]])) 
        self.action_space = Box(low=np.array([[0, 0], [0, 0]]), high=np.array([[12, 12], [8, 12]]), dtype=np.int8 )
        # observation space
        obsv =  {
        "hand": Box(high=np.array([[13, 13]*79]), dtype=np.int8,low=np.array([[-1, -1]*79])),
        "placements": Box(high=np.array([[9, 13]*208]), dtype=np.int8,low=np.array([[-1,-1]*208])),
        "trains": Box(high=np.array([[9, 2]*8]), dtype=np.int8,low=np.array([[-1,-1]*8]))
        }
        self.observation_space = Dict(obsv)
        #setup game
        self.game = Game(numPlayers)
        self.player = self.game.getPlayer(0)
        bs = BoardState.fromGame(self.game)
        handarray = [domino.sides for domino in self.player.hand]
        placements = bs.getPlacements()
        state = self.getState()
        self.state = state
        self.fails = 0
    @staticmethod
    def __padArray(array,len:int):
        return np.pad(array,((0,len),(0,0)),mode='constant',constant_values=-1)
    def getState(self):
        bs = BoardState.fromGame(self.game)
        handarray = np.array([domino.sides for domino in self.player.hand], dtype=np.int8)
        placements = np.array(bs.getPlacements(), dtype=np.int8)
        train_info = np.array([[train.id, int(train.trainUp)] for train in bs.trains], dtype=np.int8)
        
        hand_padding = DominoTrainEnv.__padArray(handarray, 79-len(handarray))
        placement_padding = DominoTrainEnv.__padArray(placements, 208-len(placements))
        train_padding = DominoTrainEnv.__padArray(train_info, 8-len(train_info))
        state =  {
            "hand": hand_padding.ravel().reshape((1,158)),
            "placements": placement_padding.ravel().reshape((1,416)),
            "trains": train_padding.ravel().reshape((1,16))
        }
        return state
    def play(self,domino:Domino, placement,player:Player, bs:BoardState):
        print(f"attempting to play {domino} on {placement} from {player}")
        game = self.game
        played = False
        if domino is not None:
            train = game.getTrain(placement[0])
            if player.play(domino,placement[1],train):
                played = True   
                if domino.isDouble:
                    print("double played")
                    players = game.players
                    start_index = players.index(player)
                    #loop through all players at "table" starting with person who played double
                    for i in range(len(players)):
                        index = (start_index + i) % len(players)
                        loop_player = players[index] 
                        newPlacement = (placement[0],domino.sides[0])
                        plays = bs.availablePlays(loop_player, placements=[placement])
                        print(f"checking if {loop_player} can play on {newPlacement}\n, plays {plays}")
                        #if player can't play on double
                        if len(plays)<=0:
                            pickupDomino = loop_player.pickup(self.game.boneyard)
                            #if a domino was actually picked up
                            if pickupDomino:
                                print(f"{loop_player} pickedup {pickupDomino}")
                                plays = bs.availablePlays(loop_player, placements=[newPlacement])
                                if len(plays)<=0: 
                                    print("player can't play pickup")
                                    loop_player.train.trainUp = True
                                    continue
                                #if possible to play pickup
                                else:
                                    print(f"attepmting to play pickup on {newPlacement}, train {train}")
                                    if loop_player.play(pickupDomino,newPlacement[1],train) is None:
                                        game.done = True
                                    break
                                    
                            # no domino was pickedup, meaning boneyard is empty and end of game
                            else:
                                game.done = True
                                break
                        #if player only has one choice to play
                        elif len(plays) == 1:
                            play = plays[0]
                            ranDomino = loop_player.getDominoFromSides(*play[0])
                            print(f"attempting to play {ranDomino} on {newPlacement}, player.train: {player.train}")
                            if loop_player.play(ranDomino,newPlacement[1],train) is None:
                                game.done = True
                            break
                        #if player has many choices to play
                        else:
                            #if ai player
                            if loop_player.id == 0:
                                print(f"letting ai make choice for double play")
                                self.game.unsastifiedDouble = (train.id,domino.sides[0])
                                self.game.prevPlayer = player.id
                                break
                            #if other players, random choice
                            else:
                                print(player.train)
                                play = plays[random.randint(0, len(plays) - 1)]
                                ranDomino = loop_player.getDominoFromSides(*play[0])
                                print(f"attempting to play {ranDomino} on {newPlacement}, player.train: {player.train}\n available plays: {plays}")
                                if loop_player.play(ranDomino,newPlacement[1],train) is None:
                                    game.done = True
                            break
            else:
                   print("Invalid Placement")
        else:
            print("Invlaid Domino")
        return played
    def maskAction(self,availablActions):
        pass
    def step(self, action):
        start_index = self.game.startingPlayer
        players = self.game.players
        length = len(players)
        stateChanged = False
        reward = 0
        bs= BoardState.fromGame(self.game)
        if bs.isValidPlay(self.player,action):
            if bs.unsastifiedDouble is not None:
                domino = action[0]
                domino = self.player.getDominoFromSides(*domino)
                self.play(domino,action[1],self.player,bs)
                reward += domino.calc_points()
                stateChanged = True
                start_index = self.game.nextPlayer(self.player.id)
                length = length - 1
                bs.unsastifiedDouble = None #probably a better way of doing this
                self.game.unsastifiedDouble = None
            if stateChanged or bs.unsastifiedDouble is None:    
                for i in range(length):
                    index = (start_index + i) % length
                    player = players[index]
                    posActions = bs.availablePlays(player)
                    #if current turn is ai
                    if player.id == 0:
                        #if no action available
                        if len(posActions)<=0:
                            self.player.pickup(self.game.boneyard)
                            posActions = bs.availablePlays(self.player)
                            if len(posActions)>0:
                                ranAction = posActions[0]
                                ranDomino = self.player.getDominoFromSides(*ranAction[0])
                                self.play(ranDomino,ranAction[1],self.player,bs)
                                stateChanged = True
                            # Check if action is valid
                        else:
                            # Apply action
                            domino = action[0]
                            domino = self.player.getDominoFromSides(*domino)
                            self.play(domino,action[1],self.player,bs)
                            reward += domino.calc_points() + 60
                            stateChanged = True
                        #invalid action   
                        
                    else:
                        print(f"current turn: {player}, plays: {posActions}")
                        if len(posActions)<=0:
                            player.pickup(self.game.boneyard)
                            posActions = bs.availablePlays(player)
                        if len(posActions)>0:
                            ranIndex = 0
                            if len(posActions)>1: ranIndex = random.randint(0, len(posActions)-1)
                            ranAction = posActions[ranIndex]
                            ranDomino = player.getDominoFromSides(*ranAction[0])
                            self.play(ranDomino,ranAction[1],player,bs)
                            stateChanged = True
        else: 
            reward += -50
            self.fails +=1
        if stateChanged:    
            self.state = self.getState()
        
        done = self.game.done
        #hard limit on game length since, the ai can make invalid plays which could loop forever
        if self.fails >=10: done = True
        elif done:
            # add negative reward for points remaining in hand at game end
            reward += -1*self.player.pointsInHand()  
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done,False, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self, seed=None):
        self.game = Game(self.game.numPlayers)
        self.player = self.game.getPlayer(0)
        self.state = self.getState()
        self.fails = 0
        return self.state, {}
    

<h2>Test ENV</h2>

<h3>Random Sample Test</h3>

In [14]:
env = DominoTrainEnv(6)

In [15]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    step = 0
    while not done:
        #env.render()
        #print(step)
        step+=1
        action = env.action_space.sample()
        n_state, reward, done, truncated, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    print(step)

Episode:1 Score:-596
10
Episode:2 Score:-663
10
Episode:3 Score:-654
10
valid play: ((1, 11), (0, 11))
attempting to play (1, 11) on [ 0 11] from id:0 train:0
current turn: id:1 train:1, plays: [((5, 11), (1, 11))]
attempting to play (5, 11) on (1, 11) from id:1 train:1
current turn: id:2 train:2, plays: []
current turn: id:3 train:3, plays: [((9, 11), (3, 11)), ((6, 11), (3, 11))]
attempting to play (9, 11) on (3, 11) from id:3 train:3
current turn: id:4 train:4, plays: [((0, 11), (4, 11)), ((3, 11), (4, 11))]
attempting to play (3, 11) on (4, 11) from id:4 train:4
current turn: id:5 train:5, plays: [((8, 11), (5, 11))]
attempting to play (8, 11) on (5, 11) from id:5 train:5
Episode:4 Score:-345
11
Episode:5 Score:-589
10
Episode:6 Score:-620
10
Episode:7 Score:-649
10
Episode:8 Score:-668
10
Episode:9 Score:-661
10
Episode:10 Score:-669
10


<h3>Env checker</h3>

In [14]:
import gymnasium as gym
from stable_baselines3.common.env_checker import check_env
env = DominoTrainEnv(6)
#print(gym.spaces.flatdim(env.observation_space))
print(env.observation_space.shape)
print(env.observation_space)
print(type(env.observation_space).__base__)
print(isinstance(env.observation_space, gym.spaces.Space))
check_env(env,skip_render_check=True)

None
Dict('hand': Box(-1, 13, (1, 158), int8), 'placements': Box(-1, [[ 9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13

<h1>Make RL Model</h1>

<h2>Intial Attempt</h2>

In [3]:
import numpy as np
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

In [5]:
env = DominoTrainEnv(6)

# Initialize the PPO agent
#model = PPO("MultiInputPolicy", env,verbose=1)

# Train the agent

# Save the trained model

# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
model = PPO.load("PPO_DominoTrain", env=env,verbose=1)
model.learn(total_timesteps=int(5e5), progress_bar=True)
model.save("PPO_DominoTrain")
model = PPO.load("PPO_DominoTrain", env=env)

# Evaluate the agent
# NOTE: If you use wrappers with your environment that modify rewards,
#       this will be reflected here. To evaluate with original rewards,
#       wrap environment in a "Monitor" wrapper before other wrappers.
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
print(mean_reward, std_reward)
# Load the saved model
#model = PPO.load("PPO_DominoTrain")

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501,698/500,000  [ 0:19:57 < 0:00:00 , 433 it/s ]

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-500.0 0.0


<h2>debugging minimal reproducable</h2>

In [18]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, MultiDiscrete, Box, Dict
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

class TestEnv(Env):
    def __init__(self,numPlayers:int):
        # Actions we can take, 13,13 for possible domino sides, [9,13] for possible domino placements
        #self.action_space = MultiDiscrete(np.array([[13, 13], [9, 13]]))
        self.action_space = Box(low=np.array([[0, 0], [0, 0]]), high=np.array([[12, 12], [8, 12]]), dtype=np.int8 )
        # observation space
        obsv =  {
        "hand": Box(high=np.array([[13, 13]*79]), dtype=np.int8,low=np.array([[-1, -1]*79]))
        }
        self.observation_space = Dict(obsv)
        #self.observation_space = Box(high=np.array([[13, 13]*79]).ravel().reshape((1,158)), dtype=np.int8,low=np.array([[-1, -1]*79]).ravel().reshape((1,158)))
        self.state = self.getState()
    @staticmethod
    def __padArray(array,len:int):
        return np.pad(array,((0,len),(0,0)),mode='constant',constant_values=-1)
    def getState(self):
        #array values are placeholders
        handarray = np.array([(11,11),(12,11)], dtype=np.int8)
        
        hand_padding = TestEnv.__padArray(handarray, 79-len(handarray))
        state =  {
            "hand": hand_padding.ravel().reshape((1,158)),
        }
        #state = state["hand"]
        return state
    def step(self, action):
        self.state = self.getState()
        # Return step information
        reward = 1
        done = False
        info = {}
        return self.state, reward, done,False, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self, seed=None):
        self.state = self.getState()
        return self.state, {}

env = TestEnv(6)
print(check_env(env,skip_render_check=True))
model = PPO("MultiInputPolicy", env)

None


C:\Users\lilmr\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\env_checker.py:439: UserWarning: Your action space has dtype int8, we recommend using np.float32 to avoid cast errors.
  warnings.warn(
